In [1]:
from lib_for_data_reading import *
from country_run import *
from perc import *
from lib_for_cc import *
import re
import numpy as np
import pandas as pd

### 1. Data loading

In [2]:
nameofthisround='spring2020'


# directories
model             = os.getcwd()
data              = model+'/data/'
scenar_folder     = model+'/scenar_def/'
finalhhdataframes = model+'/finalhhdataframes/'
ssp_folder        = model+'/ssp_data/'
pik_data          = model+'/pik_data/'
iiasa_data        = model+'/iiasa_data/'

# simulation inputs
codes         = read_csv('wbccodes2014.csv')
codes_tables  = read_csv(ssp_folder+"ISO3166_and_R32.csv")
ssp_pop       = read_csv(ssp_folder+"SspDb_country_data_2013-06-12.csv",low_memory=False)
ssp_gdp       = read_csv(ssp_folder+"SspDb_compare_regions_2013-06-12.csv")
hhcat         = read_csv(scenar_folder+"hhcat_v2.csv")
industry_list = read_csv(scenar_folder+"list_industries.csv")

present    = "{}/{}_present/".format(model,nameofthisround)
baselines  = "{}/{}_baselines/".format(model,nameofthisround)
with_cc    = "{}/{}_with_cc/".format(model,nameofthisround)
baselines_2050  = "{}/{}_baselines_2050/".format(model,nameofthisround)
with_cc_2050    = "{}/{}_with_cc_2050/".format(model,nameofthisround)

In [3]:
######################################
# Load necessary files
#
#load climate change impacts data 
# ^ This will be IIASA inputs
price_increase_reg = read_csv(iiasa_data+"boundaries_food2.csv")
food_share_data = read_csv(data+"food_shares_wbreg.csv")
#
####
# Now WITH malaria!!!
malaria_diff    = read_csv(data+"malaria_diff_2006-2035.csv")
malaria_bounds  = read_csv(data+"malaria_bounds.csv",index_col="var")
diarrhea_bounds = read_csv(data+"diarrhea_bounds.csv",index_col="var")
#
#####
diarrhea_shares  = read_csv(data+"diarrhea_who.csv",index_col="Region")
diarrhea_shares.index=diarrhea_shares.index.str.strip()
diarrhea_reg    = read_csv(data+"wbccodes2014_who.csv",index_col="country")
#
#####
# No natural disasters!
fa                       = read_csv(data+"disasters/av_fa_for_cc_model.csv",index_col='iso3')
vulnerabilities          = read_csv(data+"disasters/vulnerabilities.csv",index_col='iso3')
#
#####
# Need to see what these are, as distinct from price_increase. Probably temperature?
climate_param_bounds     = read_csv(data+"climate_param_bounds.csv",index_col="var")
climate_param_thresholds = read_csv(data+"climate_param_thresholds.csv",index_col="var")
#
#
# No Stunting!
shares_stunting = None #read_csv(data+"share_stunting_who.csv")


######################################
# Get list of available countries...
list_csv=os.listdir(finalhhdataframes)
all_surveys=dict()
for myfile in [_ for _ in list_csv if 'DS_Store' not in _ and _ != '_Final_CPI_PPP_to_be_used.csv']:
    if 'finalhhframe' in myfile:
        cc = re.search('(.*)_finalhhframe.csv', myfile).group(1)
        all_surveys[cc]=read_csv(finalhhdataframes+myfile)

In [4]:
finalhhdataframes+myfile

'C:\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateSpring2020/finalhhdataframes/_Final_CPI_PPP_to_be_used.dta'

In [5]:
####
#load codes, population
codes         = read_csv('wbccodes2014.csv')
codes_tables  = read_csv(ssp_folder+"ISO3166_and_R32.csv")
ssp_pop       = read_csv(ssp_folder+"SspDb_country_data_2013-06-12.csv",low_memory=False)
ssp_gdp       = read_csv(ssp_folder+"SspDb_compare_regions_2013-06-12.csv")
hhcat         = read_csv(scenar_folder+"hhcat_v2.csv")
industry_list = read_csv(scenar_folder+"list_industries.csv")
#
####
# Drivers of scenarios
uncertainties    = ['shareag','sharemanu','shareemp', 'shareurban',
                    'grserv','grag','grmanu','skillpserv','skillpag','skillpmanu','p','b','voice']
lhssample        = read_csv(scenar_folder+"lhs-table-500-13uncertainties.csv")
ranges = DataFrame(columns=['min','max'],index=uncertainties)

impact_scenars = read_csv(scenar_folder+"impact-scenarios-low-high.csv").tail(1)
# ^ IIASA here? 

######################################
# Packaging for country_run()
#
datalist=(hhcat,industry_list,codes_tables,lhssample,ranges,ssp_pop,ssp_gdp,food_share_data,impact_scenars,price_increase_reg,malaria_bounds, 
          diarrhea_bounds, shares_stunting, malaria_diff, diarrhea_shares, diarrhea_reg, climate_param_bounds, climate_param_thresholds,codes,fa,vulnerabilities)
#
ssp_to_test   = [5]
#

######################################
# Do the thing (TM)
#_record_keeping = read_csv('results_summary/GMD_to_finalhhframe_status.csv').set_index('country')
#for _c in ['BAU_complete','CC_complete']:
#    if _c not in _record_keeping.columns: _record_keeping[_c] = None

In [6]:
countrycode = 'IDN'
ini_year = 2011 #get_ini_year(countrycode)
data2day = 30
year = 2030
povline = 1.9

paramvar=(year,ini_year,data2day,ssp_to_test,povline)
scenar = 30
b_value = 0
switches = []
switches = ['all']
future_with_cc = True

### Check completeness

In [12]:
complete_list = []
not_complete_list = []
not_working_list = []

for country in list(all_surveys.keys()):
    print('Starting for ' + country)
    countrycode = country
    try:
        forprim_now, forprim_bau,forprim_cc, inc = country_run(countrycode,scenar,datalist,paramvar,all_surveys,
                                                               switches,b_value,future_with_cc, with_person_data=False)
        
        if len(forprim_bau) > 0:
            complete_list.append(country)
        else:
            not_complete_list.append(country)
            
    except:
        not_working_list.append(country)

Starting for AGO
27.90521240234375 19.25885009765625 1.0
Step 2 complete
2011 to 2030
Step 3 complete
Initial pop description:
         old   children  unemployed skillworkers  servworkers    agworkers  manuworkers        urban
pop  734703  1.519e+07  4.8293e+06  7.13706e+06  1.05139e+06  4.06309e+06  3.91514e+06  1.81284e+07
share employed_i: 0.6515384262634101 
share ag_i: 0.44997302703441167 
share manu_i: 0.4335886376109811 
share skilled_i: 0.5149797528422408 
share urban_i: 0.6086696184504928
SSP/population growth (2030) 30013947.56 0.01639960604566504


Fit parameter inputs (ranges):
                   min        max
shareag      0.154928   0.479329
sharemanu    0.166678    0.39474
shareemp     0.554211   0.819333
shareurban    0.70408    0.70408
grserv     -0.0267882  0.0332118
grag       -0.0267882  0.0332118
grmanu     -0.0267882  0.0332118
skillpserv     2.3362     2.3362
skillpag      1.98006    1.98006
skillpmanu    1.70963    1.70963
p               0.001        0.2
b    

Presolve started.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Optimizer terminated. Time: 0.03    


Interior-point solution summary
  Problem status  : PRIMAL_INFEASIBLE
  Solution status : PRIMAL_INFEASIBLE_CER
  Dual.    obj: 1.3342606688e+07    nrm: 1e+00    Viol.  con: 0e+00    var: 0e+00  
Starting for AUT
240.6071929931641 97.73967742919922 1.0
Step 2 complete
2011 to 2030
Step 3 complete
Initial pop description:
              old     children   unemployed skillworkers  servworkers agworkers  manuworkers        urban
pop  1.58795e+06  1.17971e+06  1.93789e+06  3.94673e+06  2.66607e+06    152040  1.12863e+06  5.34969e+06
share employed_i: 0.6706857478816277 
share ag_i: 0.03852291360636226 
share manu_i: 0.2859643341030572 
share skilled_i: 

14  3.1e-02  5.6e-12  2.1e-06  1.04e+00   1.354830612e+04   1.354830273e+04   6.9e-05  0.11  
15  6.0e-03  2.3e-11  1.7e-07  1.00e+00   1.353025627e+04   1.353025559e+04   1.3e-05  0.11  
16  7.0e-04  1.2e-10  6.8e-09  1.00e+00   1.352501244e+04   1.352501236e+04   1.5e-06  0.11  
17  8.9e-05  1.3e-09  3.1e-10  1.00e+00   1.352443046e+04   1.352443045e+04   2.0e-07  0.13  
18  1.1e-05  1.6e-08  1.4e-11  1.00e+00   1.352435594e+04   1.352435594e+04   2.5e-08  0.13  
19  1.8e-06  2.1e-07  8.7e-13  1.00e+00   1.352434659e+04   1.352434659e+04   3.8e-09  0.13  
Optimizer terminated. Time: 0.14    


Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 1.3524346563e+04    nrm: 2e+06    Viol.  con: 1e-02    var: 0e+00  
  Dual.    obj: 1.3524346613e+04    nrm: 3e-02    Viol.  con: 9e-09    var: 9e-06  
I am running climate scenario 1 of baseline 30 with ssp 5 of BDI
True True True True True
Starting for BEL
180.5696411132812

Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 7261            
  Cones                  : 0               
  Scalar variables       : 7253            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimi

2   3.2e+02  1.4e-13  1.5e+00  -5.33e-01  3.060710334e+03   2.999314801e+03   7.6e-01  0.09  
3   3.0e+02  1.2e-13  1.3e+00  1.17e-01   4.302108025e+03   4.230813909e+03   7.2e-01  0.09  
4   2.2e+02  1.5e-13  8.0e-01  2.07e-01   1.125817340e+04   1.116376910e+04   5.4e-01  0.11  
5   1.5e+02  1.2e-13  4.5e-01  4.89e-01   1.493078946e+04   1.485088161e+04   3.7e-01  0.11  
6   5.3e+01  4.0e-13  9.4e-02  6.96e-01   2.409948690e+04   2.406156278e+04   1.3e-01  0.11  
7   3.5e+01  2.7e-13  5.2e-02  9.04e-01   1.768373326e+04   1.766288050e+04   8.3e-02  0.13  
8   2.3e+01  1.2e-12  2.9e-02  9.32e-01   1.651362688e+04   1.650052055e+04   5.5e-02  0.13  
9   1.6e+01  8.7e-13  1.7e-02  9.31e-01   1.318790579e+04   1.318004714e+04   3.8e-02  0.13  
10  8.5e+00  4.2e-13  6.8e-03  9.87e-01   1.343801804e+04   1.343398536e+04   2.0e-02  0.14  
11  4.4e+00  2.5e-13  2.6e-03  9.80e-01   1.146953585e+04   1.146770145e+04   1.0e-02  0.14  
12  1.7e+00  3.0e-13  6.2e-04  1.02e+00   1.135274455e+04   

...

  Type                   : QO (quadratic optimization problem)
  Constraints            : 1514            
  Cones                  : 0               
  Scalar variables       : 1506            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.01    
Presolve started.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Optimizer terminated. Time: 0.09    


Interior-point solution summary
  Problem status  : PRIMAL_INFEASIBLE
  Solution status : PRIMAL_INFEASIBLE_CER
  Dual.    obj: 2.9246837296e+04    nrm: 1e+00    Viol.  con: 0e+00    var: 0e+00  
Starting for TUN
17.739898681640625 17.739898681640625 

Step 6.5 complete
I am running baseline 30 with ssp 5 of TZA
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 4649            
  Cones                  : 0               
  Scalar variables       : 4641            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.02            
Lin. dep.  - number                 : 0               
Presolve terminat

Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 7626              after factor           : 7626            
Factor     - dense dim.             : 0                 flops                  : 7.18e+05        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   3.9e+02  0.0e+00  2.0e+00  0.00e+00   5.000000000e-01   0.000000000e+00   1.0e+00  0.06  
1   3.7e+02  1.8e-13  1.9e+00  -8.31e-01  4.307848075e+03   4.221423281e+03   9.3e-01  0.09  
2   2.6e+02  1.3e-13  1.5e+00  -7.49e-01  7.532708577e+03   7.461355100e+03   6.6e-01  0.11  
3   2.4e+02  1.2e-13  1.4e+00  -4.94e-01  7.870787742e+03   7.801002056e+03   6.2e-01  0.11  
4   2.0e+02  1.1e-13  1.1e+00  -3.51e-01  1.125464758e+04   1.118095102e+04   5.2e-01  0.11  
5   1.4e+02  9.6e-14  6.2e-01  -2.61e-02  3.087581333e+04   3.077700883e+04   3.6e-01  0.13  
6   8.5e+01  1.1e-13  2.6e-01  6.02e-01   6.3347

  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 4311            
  Cones                  : 0               
  Scalar variables       : 4303            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 540
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 4836              conic                  : 4304            
Optimizer  - Semi-definite variables: 0                 scalarized             : 0               
Factor     - setup time             : 0.02              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 3446              after factor           : 3447   

Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 5210            
  Cones                  : 0               
  Scalar variables       : 5202            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 15
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 5210              conic                  : 5203            
Optimizer  - Semi-definite variables: 0                 scalarized             : 0               
Factor     - setup time             : 0.00  

In [13]:
len(complete_list), len(not_complete_list), len(not_working_list)

(52, 46, 5)

In [14]:
complete_list

['AGO',
 'BDI',
 'BEN',
 'BFA',
 'BGD',
 'BRA',
 'BTN',
 'BWA',
 'CIV',
 'CMR',
 'COD',
 'COM',
 'DNK',
 'DOM',
 'EST',
 'ETH',
 'FIN',
 'FRA',
 'GBR',
 'GHA',
 'GTM',
 'IRL',
 'IRQ',
 'JOR',
 'KAZ',
 'LKA',
 'LSO',
 'LTU',
 'LUX',
 'MAR',
 'MMR',
 'MOZ',
 'MRT',
 'NER',
 'NGA',
 'NIC',
 'NOR',
 'PAK',
 'PHL',
 'PNG',
 'POL',
 'SDN',
 'SLE',
 'SWE',
 'SWZ',
 'TGO',
 'THA',
 'TUN',
 'TZA',
 'UGA',
 'VNM',
 'ZMB']

In [15]:
not_complete_list

['ARG',
 'AUT',
 'BEL',
 'BGR',
 'BIH',
 'BOL',
 'CHE',
 'COG',
 'COL',
 'CRI',
 'CYP',
 'CZE',
 'DJI',
 'ECU',
 'ESP',
 'FSM',
 'GRC',
 'HND',
 'HRV',
 'HUN',
 'IDN',
 'IRN',
 'ISL',
 'ITA',
 'LAO',
 'LBN',
 'LVA',
 'MEX',
 'MKD',
 'MLT',
 'MNG',
 'NLD',
 'PAN',
 'PER',
 'PRT',
 'PSE',
 'ROU',
 'SLB',
 'SLV',
 'SVK',
 'SVN',
 'TCD',
 'TLS',
 'TON',
 'URY',
 'VUT']

In [16]:
not_working_list

['KIR', 'SSD', 'SYC', 'TUV', 'XKX']

In [17]:
import pickle

completeness_check = {'complete': complete_list, 
                      'not_complete': not_complete_list,
                      'not_working': not_working_list}

pickle.dump( completeness_check, open( "survey_completeness_check.p", "wb" ) )